# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pathlib import Path
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [8]:
readpath = Path.cwd() / 'cities.csv'
df = pd.read_csv(readpath, index_col=0)
df.head()

,Lat Guess,Long Guess,CityName,CountryCode,Temperature,Humidity,Cloudiness,Wind Speed,Lat Real,Long Real
0,-59.750558,98.501929,busselton,au,54.00,97.0,22.0,7.92,-33.65,115.33
1,-75.446240,-56.957897,ushuaia,ar,48.20,39.0,40.0,14.99,-54.80,-68.30
2,-35.554173,-122.537342,rikitea,pf,75.33,84.0,100.0,17.27,-23.12,-134.97
3,44.481512,142.016305,nayoro,jp,39.40,86.0,100.0,3.31,44.35,142.46
4,-51.126258,21.364311,bredasdorp,za,62.60,82.0,100.0,2.24,-34.53,20.04


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [10]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [13]:
locations = df[['Lat Real', 'Long Real']]
weights = df['Humidity']
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
    - 75-85 F
    - Wind Speed <8
    - 20% Cloudiness
* Drop any rows will null values.

In [22]:
# Filter a COPY of the original dataframe based on above parameters for a "perfect day"
df_copy = df.copy()
df_copy = df_copy.loc[df_copy['Temperature'] >= 75 ]
df_copy = df_copy.loc[df_copy['Temperature'] <= 85 ]
df_copy = df_copy.loc[df_copy['Wind Speed'] <= 8]
df_copy = df_copy.loc[df_copy['Cloudiness'] <= 20]
print(f'The remaining number of cities that fit our criteria is {len(df_copy)}')


The remaining number of cities that fit our criteria is 11


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [23]:
# Set up our basic query
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
